In [1]:
import csv
import io
import json
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from loguru import logger
from PIL import Image
import ray
import tensorflow as tf

num_train_shards = 64
num_val_shards = 8
ray.init()
tf.get_logger().setLevel('ERROR')

2020-11-19 21:22:26,542	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [2]:
def chunkify(l, n):
    size = len(l) // n
    start = 0
    results = []
    for i in range(n - 1):
        results.append(l[start:start + size])
        start += size
    results.append(l[start:])
    return results


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy(
        )  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [10]:
def genreate_tfexample(anno):
    filename = anno['filename']
    filepath = anno['filepath']
    with open(filepath, 'rb') as image_file:
        content = image_file.read()

    image = Image.open(filepath)
    if image.format != 'JPEG' or image.mode != 'RGB':
        image_rgb = image.convert('RGB')
        with io.BytesIO() as output:
            image_rgb.save(output, format="JPEG", quality=95)
            content = output.getvalue()

    width, height = image.size
    depth = 3

    x = [
        joint[0] / width if joint[0] >= 0 else joint[0]
        for joint in anno['joints']
    ]
    y = [
        joint[1] / height if joint[1] >= 0 else joint[0]
        for joint in anno['joints']
    ]
    # 0 - invisible, 1 - occluded, 2 - visible
    v = [0 if joint_v == 0 else 2 for joint_v in anno['joints_visibility']]

    feature = {
        'image/height':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/depth':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[depth])),
        'image/object/parts/x':
        tf.train.Feature(int64_list=tf.train.Int64List(value=list(map(int, x)))),
        'image/object/parts/y':
        tf.train.Feature(int64_list=tf.train.Int64List(value=list(map(int, y)))),
        'image/object/parts/v':
        tf.train.Feature(int64_list=tf.train.Int64List(value=list(map(int, v)))),
        'image/object/center/x':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[int(width / 2)])),
        'image/object/center/y':
        tf.train.Feature(int64_list=tf.train.Int64List(value=[int(height / 2)])),
        'image/object/scale':
        tf.train.Feature(float_list=tf.train.FloatList(value=[width * height])),
        'image/encoded':
        _bytes_feature(content),
        'image/filename':
        _bytes_feature(filename.encode())
    }

#     feature = {}
#     feature['image/height'] = tf.train.Feature(int64_list=tf.train.Int64List(value = [height]))
#     feature['image/width'] = tf.train.Feature(int64_list=tf.train.Int64List(value = [width]))
#     feature['image/depth'] = tf.train.Feature(int64_list=tf.train.Int64List(value = [depth]))
#     feature['image/object/parts/x'] = tf.train.Feature(int64_list=tf.train.Int64List(value = x))
#     feature['image/object/parts/y'] = tf.train.Feature(int64_list=tf.train.Int64List(value = y))
#     feature['image/object/parts/v'] = tf.train.Feature(int64_list=tf.train.Int64List(value = v))
#     feature['image/encoded'] = _bytes_feature(content)
#     feature['image/filename'] = _bytes_feature(filename.encode())
    
    features = tf.train.Features(feature=feature)

    return tf.train.Example(features=tf.train.Features(feature=feature))


@ray.remote
def build_single_tfrecord(chunk, path):
    print('start to build tf records for ' + path)

    with tf.io.TFRecordWriter(path) as writer:
        for anno_list in chunk:
            tf_example = genreate_tfexample(anno_list)
            writer.write(tf_example.SerializeToString())

    print('finished building tf records for ' + path)


def build_tf_records(annotations, total_shards, split):
    chunks = chunkify(annotations, total_shards)
    futures = [
        # train_0001_of_0064.tfrecords
        build_single_tfrecord.remote(
            chunk, './tfrecords/{}_{}_of_{}.tfrecords'.format(
                split,
                str(i + 1).zfill(4),
                str(total_shards).zfill(4),
            )) for i, chunk in enumerate(chunks)
    ]
    ray.get(futures)


def parse_one_annotation(anno, image_dir):
    filename = anno['image']
    joints = anno['joints']
    joints_visibility = anno['joints_vis']
    annotation = {
        'filename': filename,
        'filepath': os.path.join(image_dir, filename),
        'joints_visibility': joints_visibility,
        'joints': joints,
    }
    return annotation


# def main():
# if __name__ == '__main__':
#     main()

In [11]:
print('Start to parse annotations.')
if not os.path.exists('./tfrecords'):
    os.makedirs('./tfrecords')

with open('./mpii_human_pose_v1_u12_2/train.json') as train_json:
    train_annos = json.load(train_json)
    train_annotations = [
        parse_one_annotation(anno, './mpii/images/')
        for anno in train_annos
    ]
    print('First train annotation: ', train_annotations[0])
    del (train_annos)

with open('./mpii_human_pose_v1_u12_2/validation.json') as val_json:
    val_annos = json.load(val_json)
    val_annotations = [
        parse_one_annotation(anno, './mpii/images/') for anno in val_annos
    ]
    print('First val annotation: ', val_annotations[0])
    del (val_annos)

print('Start to build TF Records.')
build_tf_records(train_annotations, num_train_shards, 'train')
build_tf_records(val_annotations, num_val_shards, 'val')

print('Successfully wrote {} annotations to TF Records.'.format(
    len(train_annotations) + len(val_annotations)))

Start to parse annotations.
First train annotation:  {'filename': '015601864.jpg', 'filepath': './mpii/images/015601864.jpg', 'joints_visibility': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'joints': [[620.0, 394.0], [616.0, 269.0], [573.0, 185.0], [647.0, 188.0], [661.0, 221.0], [656.0, 231.0], [610.0, 187.0], [647.0, 176.0], [637.0201, 189.8183], [695.9799, 108.1817], [606.0, 217.0], [553.0, 161.0], [601.0, 167.0], [692.0, 185.0], [693.0, 240.0], [688.0, 313.0]]}
First val annotation:  {'filename': '005808361.jpg', 'filepath': './mpii/images/005808361.jpg', 'joints_visibility': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'joints': [[804.0, 711.0], [816.0, 510.0], [908.0, 438.0], [1040.0, 454.0], [906.0, 528.0], [883.0, 707.0], [974.0, 446.0], [985.0, 253.0], [982.7591, 235.9694], [962.2409, 80.0306], [869.0, 214.0], [798.0, 340.0], [902.0, 253.0], [1067.0, 253.0], [1167.0, 353.0], [1142.0, 478.0]]}
Start to build TF Records.
(pid=71026) start to build tf records for ./tf

(pid=71027) finished building tf records for ./tfrecords/train_0037_of_0064.tfrecords
(pid=71027) start to build tf records for ./tfrecords/train_0043_of_0064.tfrecords
(pid=71031) finished building tf records for ./tfrecords/train_0036_of_0064.tfrecords
(pid=71031) start to build tf records for ./tfrecords/train_0044_of_0064.tfrecords
(pid=71025) finished building tf records for ./tfrecords/train_0035_of_0064.tfrecords
(pid=71025) start to build tf records for ./tfrecords/train_0045_of_0064.tfrecords
(pid=71030) finished building tf records for ./tfrecords/train_0038_of_0064.tfrecords
(pid=71030) start to build tf records for ./tfrecords/train_0046_of_0064.tfrecords
(pid=71024) finished building tf records for ./tfrecords/train_0040_of_0064.tfrecords
(pid=71024) start to build tf records for ./tfrecords/train_0047_of_0064.tfrecords
(pid=71028) finished building tf records for ./tfrecords/train_0039_of_0064.tfrecords
(pid=71028) start to build tf records for ./tfrecords/train_0048_of_0